In [1]:
import jax
import jax.numpy as jnp
import pennylane as qml
from flax import linen as nn
from flax.training import train_state
from qlstm_ae import qlstm_autoencoder,train_model,load_model,save_model
import numpy as np



In [2]:
def generate_clean_data(n_samples, input_dim):
    return np.random.randint(0, 2, (n_samples, input_dim))
def add_noise(x, noise_level=0.2):
    noise = np.random.randn(*x.shape) < noise_level
    return (x + noise) % 2  # Flip bits
def binary_accuracy(original, reconstructed, threshold=0.5):
    pred = (reconstructed > threshold).float()
    correct = (pred == original).float()
    return correct.mean().item()

In [3]:
import jax_dataloader as jdl
noise_level=0.3
input_size=8
batch_size=16
samples=batch_size*300
clean_data = generate_clean_data(samples, input_size)
noisy_data = add_noise(clean_data, noise_level=noise_level)
#add an extra dimension for features  at the end of clean data and noisy data
clean_data = np.expand_dims(clean_data, axis=-1)
noisy_data = np.expand_dims(noisy_data, axis=-1)
train_data=jdl.ArrayDataset(clean_data,noisy_data)
train_loader=jdl.DataLoader(train_data,backend='jax',batch_size=16,shuffle=True,drop_last=True)
test_samples = generate_clean_data(96, input_size)
noisy_test = add_noise(test_samples, noise_level=0.3)
test_data=jdl.ArrayDataset(test_samples,noisy_test)
test_loader=jdl.DataLoader(test_data,backend='jax',batch_size=16,shuffle=False,drop_last=True)
train_model(input_size,train_loader,test_loader,batch_size,epochs=10)

Epoch 0, Loss: 0.25551795959472656
Epoch 1, Loss: 0.25169360637664795
Epoch 2, Loss: 0.2531997561454773
Epoch 3, Loss: 0.25744932889938354
Epoch 4, Loss: 0.2557903528213501
Epoch 5, Loss: 0.25356292724609375
Epoch 6, Loss: 0.25330451130867004
Epoch 7, Loss: 0.25358209013938904
Epoch 8, Loss: 0.25291502475738525
Epoch 9, Loss: 0.25361189246177673


(qlstm_autoencoder(
     # attributes
     seq_lenght = 8
     n_qlayers = 1
     n_qubits = 6
     hidden_size = 64
     target_size = 1
 ),
 {'params': {'Dense_0': {'bias': Array([ 0.6331002,  0.6062814, -0.7819197, -2.4399712,  1.1148129,
            1.0047803], dtype=float32),
    'kernel': Array([[ 2.24877670e-01,  3.45999524e-02,  1.07176103e-01,
            -1.64086074e-01,  1.12823166e-01, -3.52363996e-02],
           [ 1.03669472e-01,  2.22866628e-02, -3.54889892e-02,
            -3.42498603e-03, -2.68918648e-02, -1.44831941e-01],
           [-1.24589220e-01,  2.40041083e-03, -1.13036774e-01,
            -6.72364756e-02, -1.12042874e-01, -1.93510115e-01],
           [-1.66665569e-01, -2.07589865e-01,  1.45077169e-01,
             1.40347213e-01,  2.78312620e-02,  1.52501523e-01],
           [ 2.58644551e-01,  1.44028738e-01, -1.48174763e-01,
            -1.23371802e-01, -1.03151046e-01,  1.05255708e-01],
           [ 1.18035406e-01,  3.18903588e-02, -1.58121347e-01,
          

In [4]:
import jax.numpy as jnp
import pickle

# Cargar datos de prueba
test_samples = generate_clean_data(96, input_size)
noisy_test = add_noise(test_samples, noise_level=0.3)
noisy_test = np.expand_dims(noisy_test, axis=-1)  # Añade dimensión de característica

# Cargar modelo y pesos
model_class = qlstm_autoencoder
input_shape = noisy_test.shape  # (batch, seq_len, 1)
model, params = load_model(model_class, input_shape=input_shape)

# Convertir datos a jax.numpy
noisy_test_jax = jnp.array(noisy_test)

# Realizar predicción
predictions = model.apply(params, noisy_test_jax)

# Mostrar resultados
print("Predicciones shape:", predictions.shape)
print("Predicciones (primeras 5):", predictions[:5])


TypeError: qlstm_autoencoder.__init__() got an unexpected keyword argument 'input_size'